<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
with open("PHALP_AAaron_1_features.pkl", "rb") as f:
  data = pickle.load(f)

In [ ]:
import torch
import torch.nn as nn

class SkiSwingLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.3
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 wegen bidirectional

    def forward(self, x):
        output, (hidden, _) = self.lstm(x)
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        return self.fc(self.dropout(hidden))

In [ ]:
class FrameWiseLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True, bidirectional=True, dropout=0.3)
    self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(hidden_size * 2, num_classes)

  def forward(self, x):
    lstm_out, _ = self.lstm(x)
    return self.fc(self.dropout(lstm_out))

In [ ]:
import torch
import torch.serialization
torch.serialization.add_safe_globals([SkiSwingLSTM])

model_schwung = torch.load("ski_schwung_classifier.pt", weights_only=False, map_location=torch.device('cpu'))
model_left_right_classifier = torch.load("left_right_classifier.pt", weights_only=False, map_location=torch.device('cpu'))


In [ ]:
model_schwung.eval()
model_left_right_classifier.eval()

FrameWiseLSTM(
  (lstm): LSTM(37, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=3, bias=True)
)

In [ ]:
sequence_data = []

for frame_idx in data["schwung_labels"]:
    try:
        frame_features = []

        frame_features.append(data["COM_to_ground"][frame_idx])
        frame_features.append(data["knee_angles_right"][frame_idx])
        frame_features.append(data["knee_angles_left"][frame_idx])

        for axis in data["joint_angles"][frame_idx]:
            frame_features.extend(data["joint_angles"][frame_idx][axis].values())

        for axis in data["axis_angles"][frame_idx]:
            frame_features.extend(data["axis_angles"][frame_idx][axis].values())

        frame_features.extend(data["COM_angles"][frame_idx].values())

        sequence_data.append(frame_features)

    except KeyError as e:
        print(f"Fehler bei Frame {frame_idx}: {e}")
        continue


In [ ]:
x = torch.tensor(sequence_data, dtype=torch.float32).unsqueeze(0)  # Shape: [1, seq_len, input_dim]


In [ ]:
# Vorhersage
with torch.no_grad():
    pred_style = model_schwung(x).argmax(dim=1).item()

# Ausgabe
print("Fahrstil:", pred_style)

Fahrstil: 2


In [ ]:
sequence_data = []

for frame_idx in data["schwung_labels"]:
    try:
        frame_features = []

        frame_features.append(data["COM_to_ground"][frame_idx])
        frame_features.append(data["knee_angles_right"][frame_idx])
        frame_features.append(data["knee_angles_left"][frame_idx])

        for axis in data["joint_angles"][frame_idx]:
            frame_features.extend(data["joint_angles"][frame_idx][axis].values())

        for axis in data["axis_angles"][frame_idx]:
            frame_features.extend(data["axis_angles"][frame_idx][axis].values())

        frame_features.extend(data["COM_angles"][frame_idx].values())
        frame_features.append(pred_style)

        sequence_data.append(frame_features)

    except KeyError as e:
        print(f"Fehler bei Frame {frame_idx}: {e}")
        continue


# 📦 In Tensor umwandeln
x_turn = torch.tensor(sequence_data, dtype=torch.float32).unsqueeze(0)  # [1, seq_len, 37]


In [ ]:
# Vorhersage
with torch.no_grad():
    pred_turn = model_left_right_classifier(x_turn).argmax(dim=-1).squeeze().numpy()


# Ausgabe
print("Links/Rechts (pro Frame):", pred_turn)


Links/Rechts (pro Frame): [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 2]


In [ ]:
import cv2

# Klassenliste (falls le.classes_ nicht verfügbar ist)
class_names = ['CL', 'EKK', 'PD', 'PDK', 'PDL', 'PGK', 'PGL', 'PS']

# Eingabevideo laden
video_path = "PHALP_AAaron_1.mp4"
cap = cv2.VideoCapture(video_path)

# VideoWriter vorbereiten
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("annotated_output.mp4", fourcc, fps, (width, height))

frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame_idx >= len(pred_turn):
        break

    # Schwungrichtung
    swing_label = "Linksschwung" if pred_turn[frame_idx] == 1 else "Rechtsschwung"

    # Fahrstil
    style_label = class_names[pred_style]

    # Text oben links (Schwung)
    cv2.putText(frame, swing_label, (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    # Text unten links (Fahrstil)
    cv2.putText(frame, f"Fahrstil: {style_label}", (50, height - 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
